In [ ]:
import pandas as pd
import numpy as np
import crawlerdetect as cd
import seaborn as sns
import datetime

df = pd.read_csv('logs1.csv')
origdf = df

df

### Exclude bots

In [ ]:
crawler_detect = cd.CrawlerDetect()

df['crawler'] = np.vectorize(crawler_detect.isCrawler)(df.useragent)

bots = df[(df.crawler == True)]

df = df[(df.crawler == False) & (df.useragent != '-')]

df

### Limit Methods to GET/POST

In [ ]:
df = df[(df.method == 'GET') | (df.method == 'POST')]

df

### How many different unique IPs with at least one request?

In [ ]:
v = df.ip.value_counts()
single_requests_df = df.drop_duplicates(subset=['ip'])

single_requests_df

In [ ]:
print('There are ' + str(len(single_requests_df.index)) + ' unique IPs with at least one request')

### How many different unique IPs with at least two requests?

In [ ]:
v = df.ip.value_counts()
multiple_requests_df = df[df.ip.isin(v.index[v.gt(1)])]
multiple_requests_df = multiple_requests_df.drop_duplicates(subset=['ip'])

multiple_requests_df

In [ ]:
print('There are ' + str(len(multiple_requests_df.index)) + ' unique IPs with at least two requests')

### How many unique IPs with requests on more than one day? (returning users)?

In [ ]:
# TODO
v = df.ip.value_counts()
returning_requests_df = df[df.ip.isin(v.index[v.gt(1)])]
returning_requests_df = returning_requests_df.drop_duplicates(subset=['ip'])

returning_requests_df

In [ ]:
print('There are ' + str(len(returning_requests_df.index)) + ' unique IPs with requests on more than one day')

### Browser-looking requests per week histogram

In [ ]:
def line_format(label):
    month = label.month_name()[:3]
    if month == 'Jan':
        month += f'\n{label.year}'
    return month

df['computed_date'] = df.date.astype(str) + ' ' + df.month.astype(str) + ' ' + df.year.astype(str)

df['computed_date'] = df['computed_date'].astype('datetime64[ns]')

plot = df[['computed_date','ip']]

plot.set_index('computed_date', inplace=True)

plot.resample('1W').count().plot(kind='bar')

### Evidence for people writing code against it (e.g. user agent of  "Python" or r studio" or something)

In [ ]:
code_strings = '|'.join(['python','r studio'])

df_code = origdf[origdf['useragent'].str.contains(code_strings,case=False)]

df_code


### Evidence for people downloading the data

In [ ]:
download_strings = '|'.join(['.xlsx'])

df_downloads = df[df['request'].str.contains(download_strings, case=False)]

df_downloads